In [1]:
import pandas as pd
import json

In [2]:
from elo import Elo
elo = Elo()

In [3]:
boards = [{1: [1, 2, 3, 4], 2: [4, 3, 2, 1]},
          {1: [1, 2, 3, 4], 2: [3, 4, 1, 2]},
          {1: [1, 2, 3, 4], 2: [2, 1, 4, 3]},
          {1: [1, 2, 3, 4], 2: [1, 2, 3, 4]}]

In [4]:
date = '2018-02-14'

In [5]:
df = pd.DataFrame(
    [{'Day': date,
      'Round': rnd + 1,
      'Board offset': abs((rnd % 2 - 1) + (i // 2)),  # If board offset is even then player 1 is white
      'Team 1 division': division['division'],
      'Team 1 full name': f"{divisionList['name'].strip()} {divisionList['surname'].strip()}",
      'Board 1': boards[rnd][1][i],
      'Player 1': f"{player1['name'].strip()}",
      'Rating 1': int(f"{player1['rating'].strip()}"),
      'Team 2 division': division['division'],
      'Team 2 full name': f"{divisionList['name2'].strip()} {divisionList['surname2'].strip()}",
      'Board 2': boards[rnd][2][i],
      'Player 2': f"{player2['name'].strip()}",
      'Rating 2': int(f"{player2['rating'].strip()}")
     }
     for division in json.load(open(f'{date}.json'))
     for divisionList in division['divisionList']
     for rnd, rounds in enumerate(divisionList['rounds'])
     for i, (player1, player2) in enumerate(zip(rounds['team1'], rounds['team2']))])

In [6]:
def score(x):
    # Player 1 is white on board 1 and 3 (board offset 0 and 2)
    p = x['Board offset'] % 2 == 0
    white_elo = x['Rating 1'] if p else x['Rating 2']
    black_elo = x['Rating 2'] if p else x['Rating 1']
    pr = elo.probabilities(white_elo, black_elo)
    return pd.Series({'Player 1 score': (pr['white'] if p else pr['black']) + pr['draw'] / 2,
                      'Player 2 score': (pr['black'] if p else pr['white']) + pr['draw'] / 2})

score_df = pd.concat([df, df.apply(score, axis=1)], axis=1)

In [7]:
# Sum score by player (+ day, division, board, team)
def f(i):
    return lambda x: pd.Series(
        {'Day': x['Day'],
         'Division': x[f'Team {i} division'],
         'Matchup': '{Team 1 full name} v {Team 2 full name}'.format(**x),
         'Board': x[f'Board {i}'],
         'Team': x[f'Team {i} full name'],
         'Player': x[f'Player {i}'],
         'Rating': x[f'Rating {i}'],
         'Score': x[f'Player {i} score']})    

total_df = pd.concat([score_df.apply(f(1), axis=1), score_df.apply(f(2), axis=1)]) \
    .groupby(['Day', 'Division', 'Matchup', 'Board', 'Team', 'Player']).agg({'Rating': 'mean', 'Score': 'sum'})

In [8]:
# Likely match scores
total_df.groupby(['Day', 'Division', 'Matchup', 'Team']).agg({'Rating': 'mean', 'Score': 'sum'})

Rating  \
Day        Division          Matchup                                        Team                                
2018-02-14 Atlantic Division Miami Champions v Buenos Aires Krakens         Buenos Aires Krakens      2428.50   
                                                                            Miami Champions           2424.00   
                             Minnesota Blizzard v Webster Windmills         Minnesota Blizzard        2482.50   
                                                                            Webster Windmills         2534.00   
                             Pittsburgh Pawngrabbers v Montclair Sopranos   Montclair Sopranos        2498.00   
                                                                            Pittsburgh Pawngrabbers   2473.00   
                             Saint Louis Arch Bishops v Montreal Chessbrahs Montreal Chessbrahs       2496.50   
                                                                            Saint Louis Arch Bishops  2499.25   
           Central Division  Ljubljana Turtles v Amsterdam Mosquitoes       Amsterdam Mosquitoes      2476.00   
                                                                            Ljubljana Turtles         2468.25   
                             London Lions v London Towers                   London Lions              2438.00   
                                                                            London Towers             2444.25   
                             Marseille Migraines v Stockholm Snowballs      Marseille Migraines       2476.00   
                                                                            Stockholm Snowballs       2510.75   
                             Reykjavik Puffins v Cannes Blockbusters        Cannes Blockbusters       2469.00   
                                                                            Reykjavik Puffins         2487.50   
           Eastern Division  Norway Gnomes v Armenia Eagles                 Armenia Eagles            2481.25   
                                                                            Norway Gnomes             2452.50   
                             Oslo Trolls v Estonia Horses                   Estonia Horses            2455.50   
                                                                            Oslo Trolls               2463.25   
                             Riga Magicians v Delhi Dynamite                Delhi Dynamite            2530.75   
                                                                            Riga Magicians            2499.50   
                             Volga Stormbringers v Mumbai Movers            Mumbai Movers             2508.00   
                                                                            Volga Stormbringers       2493.50   
           Pacific Division  Australia Kangaroos v Dallas Destiny           Australia Kangaroos       2498.75   
                                                                            Dallas Destiny            2468.25   
                             Chengdu Pandas v San Jose Hackers              Chengdu Pandas            2474.75   
                                                                            San Jose Hackers          2485.50   
                             Las Vegas Desert Rats v San Diego Surfers      Las Vegas Desert Rats     2473.50   
                                                                            San Diego Surfers         2489.50   
                             Rio Grande Ospreys v Seattle Sluggers          Rio Grande Ospreys        2495.75   
                                                                            Seattle Sluggers          2499.25   

                                                                                                         Score  
Day        Division          Matchup                                        Team                                
2018-02-14 Atlantic Division Miami

In [9]:
# Top scoring players per board
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False).apply(lambda x: x.sort_values('Score', ascending=False).head(1))

Rating  \
Day        Division          Matchup                                        Board Team                     Player                           
2018-02-14 Atlantic Division Minnesota Blizzard v Webster Windmills         1     Webster Windmills        Le Quang Liem             2739   
                             Saint Louis Arch Bishops v Montreal Chessbrahs 2     Saint Louis Arch Bishops Varuzhan Akobian          2662   
                                                                            3     Saint Louis Arch Bishops Yaroslav Zherebukh        2621   
                             Miami Champions v Buenos Aires Krakens         4     Buenos Aires Krakens     German Della Morte        2362   
           Central Division  Marseille Migraines v Stockholm Snowballs      1     Marseille Migraines      Maxime Vachier-Lagrave    2804   
                                                                            2     Stockholm Snowballs      Georg Meier               2651   
                             Ljubljana Turtles v Amsterdam Mosquitoes       3     Ljubljana Turtles        Dusko Pavasovic           2548   
                             Reykjavik Puffins v Cannes Blockbusters        4     Reykjavik Puffins        Throstur Thorhallsson     2420   
           Eastern Division  Volga Stormbringers v Mumbai Movers            1     Mumbai Movers            Viswanathan Anand         2794   
                                                                            2     Mumbai Movers            Vidit Gujrathi            2702   
                             Norway Gnomes v Armenia Eagles                 3     Norway Gnomes            Evgeny Romanov            2626   
                             Oslo Trolls v Estonia Horses                   4     Oslo Trolls              Johannes Haug             2379   
           Pacific Division  Chengdu Pandas v San Jose Hackers              1     San Jose Hackers         Shakhriyar Mamedyarov     2797   
                                                                            2     San Jose Hackers         Daniel Naroditsky         2626   
                                                                            3     San Jose Hackers         Zviad Izoria              2601   
                             Australia Kangaroos v Dallas Destiny           4     Australia Kangaroos      Temur Kuybokarov          2468   

                                                                                                                                      Score  
Day        Division          Matchup                                        Board Team                     Player                            
2018-02-14 Atlantic Division Minnesota Blizzard v Webster Windmills         1     Webster Windmills        Le Quang Liem           3.252309  
                             Saint Louis Arch Bishops v Montreal Chessbrahs 2     Saint Louis Arch Bishops Varuzhan Akobian        2.760811  
                                                                            3     Saint Louis Arch Bishops Yaroslav Zherebukh      2.586388  
                             Miami Champions v Buenos Aires Krakens         4     Buenos Aires Krakens     German Della Morte      1.657258  
           Central Division  Marseille Migraines v Stockholm Snowballs      1     Marseille Migraines      Maxime Vachier-Lagrave  3.231822  
                                                                            2     Stockholm Snowballs      Georg Meier             2.456193  
                             Ljubljana Turtles v Amsterdam Mosquitoes       3     Ljubljana Turtles        Dusko Pavasovic         2.342837  
                             Reykjavik Puffins v Cannes Blockbusters        4     Reykjavik Puffins        Throstur Thorhallsson   1.739819  
           Eastern Division  Volga Stormbringers v Mumbai Movers            1     Mumbai Movers            Viswanathan Anand       3.266296  
                              

In [10]:
# Top players by rating
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False).apply(lambda x: x.sort_values('Rating', ascending=False).head(1))

Rating  \
Day        Division          Matchup                                        Board Team                     Player                           
2018-02-14 Atlantic Division Minnesota Blizzard v Webster Windmills         1     Webster Windmills        Le Quang Liem             2739   
                             Saint Louis Arch Bishops v Montreal Chessbrahs 2     Saint Louis Arch Bishops Varuzhan Akobian          2662   
                                                                            3     Saint Louis Arch Bishops Yaroslav Zherebukh        2621   
                             Minnesota Blizzard v Webster Windmills         4     Minnesota Blizzard       Sean Nagle                2432   
           Central Division  Marseille Migraines v Stockholm Snowballs      1     Marseille Migraines      Maxime Vachier-Lagrave    2804   
                                                                            2     Stockholm Snowballs      Georg Meier               2651   
                                                                            3     Marseille Migraines      Jules Moussard            2571   
                             Reykjavik Puffins v Cannes Blockbusters        4     Reykjavik Puffins        Throstur Thorhallsson     2420   
           Eastern Division  Volga Stormbringers v Mumbai Movers            1     Mumbai Movers            Viswanathan Anand         2794   
                                                                            2     Mumbai Movers            Vidit Gujrathi            2702   
                             Norway Gnomes v Armenia Eagles                 3     Norway Gnomes            Evgeny Romanov            2626   
                             Oslo Trolls v Estonia Horses                   4     Oslo Trolls              Johannes Haug             2379   
           Pacific Division  Chengdu Pandas v San Jose Hackers              1     San Jose Hackers         Shakhriyar Mamedyarov     2797   
                                                                            2     San Jose Hackers         Daniel Naroditsky         2626   
                                                                            3     San Jose Hackers         Zviad Izoria              2601   
                             Australia Kangaroos v Dallas Destiny           4     Australia Kangaroos      Temur Kuybokarov          2468   

                                                                                                                                      Score  
Day        Division          Matchup                                        Board Team                     Player                            
2018-02-14 Atlantic Division Minnesota Blizzard v Webster Windmills         1     Webster Windmills        Le Quang Liem           3.252309  
                             Saint Louis Arch Bishops v Montreal Chessbrahs 2     Saint Louis Arch Bishops Varuzhan Akobian        2.760811  
                                                                            3     Saint Louis Arch Bishops Yaroslav Zherebukh      2.586388  
                             Minnesota Blizzard v Webster Windmills         4     Minnesota Blizzard       Sean Nagle              1.509110  
           Central Division  Marseille Migraines v Stockholm Snowballs      1     Marseille Migraines      Maxime Vachier-Lagrave  3.231822  
                                                                            2     Stockholm Snowballs      Georg Meier             2.456193  
                                                                            3     Marseille Migraines      Jules Moussard          2.263175  
                             Reykjavik Puffins v Cannes Blockbusters        4     Reykjavik Puffins        Throstur Thorhallsson   1.739819  
           Eastern Division  Volga Stormbringers v Mumbai Movers            1     Mumbai Movers            Viswanathan Anand       3.266296  
                              

In [11]:
# Top players by rating *and* score
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False) \
    .apply(lambda x: (x['Rating'].rank(ascending=False) + x['Score'].rank(ascending=False)).sort_values().head(1)) \
    .to_frame('Rating and score')

Rating and score
Day        Division          Matchup                                        Board Team                     Player                                  
2018-02-14 Atlantic Division Minnesota Blizzard v Webster Windmills         1     Webster Windmills        Le Quang Liem                        2.0
                             Saint Louis Arch Bishops v Montreal Chessbrahs 2     Saint Louis Arch Bishops Varuzhan Akobian                     2.0
                                                                            3     Saint Louis Arch Bishops Yaroslav Zherebukh                   2.0
                             Minnesota Blizzard v Webster Windmills         4     Minnesota Blizzard       Sean Nagle                           3.0
           Central Division  Marseille Migraines v Stockholm Snowballs      1     Marseille Migraines      Maxime Vachier-Lagrave               2.0
                                                                            2     Stockholm Snowballs      Georg Meier                          2.0
                             Ljubljana Turtles v Amsterdam Mosquitoes       3     Ljubljana Turtles        Dusko Pavasovic                      3.0
                             Reykjavik Puffins v Cannes Blockbusters        4     Reykjavik Puffins        Throstur Thorhallsson                2.0
           Eastern Division  Volga Stormbringers v Mumbai Movers            1     Mumbai Movers            Viswanathan Anand                    2.0
                                                                            2     Mumbai Movers            Vidit Gujrathi                       2.0
                             Norway Gnomes v Armenia Eagles                 3     Norway Gnomes            Evgeny Romanov                       2.0
                             Oslo Trolls v Estonia Horses                   4     Oslo Trolls              Johannes Haug                        2.0
           Pacific Division  Chengdu Pandas v San Jose Hackers              1     San Jose Hackers         Shakhriyar Mamedyarov                2.0
                                                                            2     San Jose Hackers         Daniel Naroditsky                    2.0
                                                                            3     San Jose Hackers         Zviad Izoria                         2.0
                             Australia Kangaroos v Dallas Destiny           4     Australia Kangaroos      Temur Kuybokarov                     2.0

In [12]:
# Tie breaker
total_df.groupby(['Day', 'Team']).agg({'Rating': 'mean', 'Score': 'sum'}).sort_values('Score', ascending=False)

Rating     Score
Day        Team                                       
2018-02-14 Webster Windmills         2534.00  9.004073
           Delhi Dynamite            2530.75  8.697006
           Saint Louis Arch Bishops  2499.25  8.627426
           San Jose Hackers          2485.50  8.549099
           Australia Kangaroos       2498.75  8.494526
           Montclair Sopranos        2498.00  8.451815
           Reykjavik Puffins         2487.50  8.352684
           San Diego Surfers         2489.50  8.308952
           Norway Gnomes             2452.50  8.204178
           Mumbai Movers             2508.00  8.192169
           Ljubljana Turtles         2468.25  8.096861
           Seattle Sluggers          2499.25  8.068263
           Miami Champions           2424.00  8.041768
           London Towers             2444.25  8.031898
           Estonia Horses            2455.50  8.017324
           Stockholm Snowballs       2510.75  8.006392
           Marseille Migraines       2476.00  7.993608
           Oslo Trolls               2463.25  7.982676
           London Lions              2438.00  7.968102
           Buenos Aires Krakens      2428.50  7.958232
           Rio Grande Ospreys        2495.75  7.931737
           Amsterdam Mosquitoes      2476.00  7.903139
           Volga Stormbringers       2493.50  7.807831
           Armenia Eagles            2481.25  7.795822
           Las Vegas Desert Rats     2473.50  7.691048
           Cannes Blockbusters       2469.00  7.647316
           Pittsburgh Pawngrabbers   2473.00  7.548185
           Dallas Destiny            2468.25  7.505474
           Chengdu Pandas            2474.75  7.450901
           Montreal Chessbrahs       2496.50  7.372574
           Riga Magicians            2499.50  7.302994
           Minnesota Blizzard        2482.50  6.995927

In [13]:
# Example matchup
total_df[total_df.index.get_level_values('Matchup') == 'Chengdu Pandas v Australia Kangaroos']

,,,,,,Rating,Score
Day,Division,Matchup,Board,Team,Player,,


In [14]:
score_df[(score_df['Team 1 full name'] == 'Chengdu Pandas') | (score_df['Team 2 full name'] == 'Chengdu Pandas')]

,Board 1,Board 2,Board offset,Day,Player 1,Player 2,Rating 1,Rating 2,Round,Team 1 division,Team 1 full name,Team 2 division,Team 2 full name,Player 1 score,Player 2 score
240,1,4,1,2018-02-14,Wang Yue,Vinesh Ravuri,2699,1918,1,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.996661,0.003339
241,2,3,1,2018-02-14,Ju Wenjun,Zviad Izoria,2574,2601,1,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.429165,0.570835
242,3,2,0,2018-02-14,Xu Xiangyu,Daniel Naroditsky,2549,2626,1,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.423675,0.576325
243,4,1,0,2018-02-14,Xiao Junyuan,Shakhriyar Mamedyarov,2077,2797,1,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.006227,0.993773
244,1,3,0,2018-02-14,Wang Yue,Zviad Izoria,2699,2601,2,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.661478,0.338522
245,2,4,0,2018-02-14,Ju Wenjun,Vinesh Ravuri,2574,1918,2,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.989748,0.010252
246,3,1,1,2018-02-14,Xu Xiangyu,Shakhriyar Mamedyarov,2549,2797,2,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.179571,0.820429
247,4,2,1,2018-02-14,Xiao Junyuan,Daniel Naroditsky,2077,2626,2,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.025618,0.974382
248,1,2,1,2018-02-14,Wang Yue,Daniel Naroditsky,2699,2626,3,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.569460,0.430540
249,2,1,1,2018-02-14,Ju Wenjun,Shakhriyar Mamedyarov,2574,2797,3,Pacific Division,Chengdu Pandas,Pacific Division,San Jose Hackers,0.202422,0.797578
